In [1]:
import pandas as pd
import re

In [2]:
pro_df = pd.read_csv('./크롤링 프로그램/programmers.csv')
cat_df = pd.read_csv('./크롤링 프로그램/catch_result.csv', index_col=0)
dart_df = pd.read_csv('./크롤링 프로그램/dart_result.csv')
incru_df = pd.read_csv('./크롤링 프로그램/incruit_result.csv')

## 1. 프로그래머스 전처리
- 주소: 시(도), 구
- 사원수: 숫자만, int타입
- 직급: 최소단위 기준, int타입

In [3]:
# 주소 전처리
address_list = []

for ad in pro_df['주소']:
    # 우편번호 패턴을 찾아서 제거
    ad = re.sub('^\d{5}\s', '', ad).strip()
    # 글자 변경
    if '대한민국' in ad:
        ad = re.sub('대한민국', '', ad).strip()
    elif '서울' in ad and '특별시' not in ad:
        ad = re.sub('서울','서울특별시',ad).strip()
    elif '경기' in ad and '경기도' not in ad:
        ad = re.sub('경기','경기도',ad).strip()
    elif '인천' in ad and '광역시' not in ad:
        ad = re.sub('인천','인천광역시',ad).strip()

    address_list.append(ad)

# 구,시,도로 끝나는 단어 리스트 정렬
add_list = [' '.join(re.findall(r'\b(\w+[구시도])\b', address)) for address in address_list]

pro_df['주소'] = add_list

In [4]:
# 사원수 전처리
employee_list = []

# '명' 및 공백 제거
for emp in pro_df['사원수']:
    employee_list.append(int(re.sub('명','',emp).strip()))

pro_df['사원수'] = employee_list
pro_df['사원수'] = pro_df['사원수'].astype(int)

In [5]:
# 직급 전처리
career_list = []

for car in pro_df['직급']:
    if '무관' not in car:  # 무관이 없는 경우 최소단위
        career_list.append(car.split('~')[0].strip())

    elif '신입' in car:  # 신입인 경우 0
        career_list.append(0)
                           
    else:  # 무관이 있는 경우 0
        career_list.append(0) 

pro_df['직급'] = career_list
pro_df['직급'] = pro_df['직급'].astype(int)  # int타입 변경

In [6]:
pro_df.to_csv('./1차 전처리.csv')
pro_df.to_excel('./1차 확인.xlsx')

## 2. 캐치 전처리
- 매출액: 만의 단위로, int타입
- 사원수: 숫자만, int타입
- 설립연도: 연도만, int타입
- 

In [27]:
cat_df

,기업명,기업형태,매출액,사원수,업종분류,설립연도
0,AB180,중소기업,272억 (2022.12),-명,응용 소프트웨어 개발 및 공급업,2015년 11월 04일
0,레몬베이스,중소기업,-,-명,응용 소프트웨어 개발 및 공급업,2020년 10월 30일
0,오토위니,대기업,"연결 재무제표 : 2조 7,545억\n별도 재무제표 : 2조 2,408억","연결 재무제표 : 1,424억\n별도 재무제표 : 1,244억","연결 재무제표 : 1,162억\n별도 재무제표 : 1,066억",2000년 04월 10일
0,페이타랩,중소기업,5억 (2021.12),52명 (2022.07),시스템 소프트웨어 개발 및 공급업,2018년 02월 28일
0,한화비전,대기업,"8,041억 (2022.12)","1,136억 (2022.12)",892억 (2022.12),2018년 04월 02일
0,발란,중소기업,891억 (2022.12),122명 (2022.12),전자상거래 소매 중개업,2015년 05월 14일
0,플라잉캣,중소기업,-,양호 (2021.01),18명 (2022.10),2018년 02월 27일
0,AB180,중소기업,272억 (2022.12),-명,응용 소프트웨어 개발 및 공급업,2015년 11월 04일
0,월급쟁이부자들,중소기업,118억 (2021.12),82억 (2021.12),66억 (2021.12),2018년 03월 27일
0,앤트,중견기업,194억 (2022.12),17억 (2022.12),14억 (2022.12),1996년 01월 16일


In [15]:
cat_df['매출액']

0                              272억 (2022.12)
0                                           -
0    연결 재무제표 : 2조 7,545억\n별도 재무제표 : 2조 2,408억
0                                5억 (2021.12)
0                            8,041억 (2022.12)
0                              891억 (2022.12)
0                                           -
0                              272억 (2022.12)
0                              118억 (2021.12)
0                              194억 (2022.12)
0                               47억 (2021.12)
0                              196억 (2022.12)
0                              355억 (2022.12)
0                               31억 (2021.12)
0                                           -
0                                           -
0                                           -
0                                           -
0                                           -
0                              191억 (2022.12)
0                                           -
0                                 

In [26]:
list = []

for pay in cat_df['매출액']:
    # 결측치 만들기
    if pay == '-':
        list.append(None)    
    else:
        list.append(pay)
    # 

list

['272억 (2022.12)',
 None,
 '연결 재무제표 : 2조 7,545억\n별도 재무제표 : 2조 2,408억',
 '5억 (2021.12)',
 '8,041억 (2022.12)',
 '891억 (2022.12)',
 None,
 '272억 (2022.12)',
 '118억 (2021.12)',
 '194억 (2022.12)',
 '47억 (2021.12)',
 '196억 (2022.12)',
 '355억 (2022.12)',
 '31억 (2021.12)',
 None,
 None,
 None,
 None,
 None,
 '191억 (2022.12)',
 None,
 None,
 None,
 '47억 (2021.12)',
 '148억 (2022.12)',
 '82억 (2022.12)',
 '82억 (2022.12)',
 None,
 None,
 '8억 (2021.12)']

# 3. 다트 전처리

# 4. 인크루트 전처리